In [6]:
import os
import torch
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from typing import Any, Dict, Type, cast
from sharp_dataloader import GenMARSH
from torch.utils.data import DataLoader
from os.path import dirname as up
import pandas as pd
import smp_metrics
from torchmetrics import Accuracy, JaccardIndex, MetricCollection
from omegaconf import OmegaConf

import numpy as np
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from MarshModel import MarshModel
from torchvision import transforms
import itertools 
import tqdm
import time

path_cur = os.path.abspath(os.getcwd())
root_path = up(path_cur)
data_dir = os.path.join(root_path, 'data', 'NAIP')

In [7]:
import sys
sys.path.append(root_path)
from utils import metrics


In [ ]:

# tmi_file = os.path.join(data_dir, 'NAIP_data_highlow_50000.csv') #NAIP_data_highlow_512.csv
# df = pd.read_csv(tmi_file)

# data_transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Resize((512, 512))])

# dataset = GenMARSH(tmi_file, transform=None, normalization=True, ndvi=False, ndwi=False, datasource='naip')

# train_size = int(0.8 * len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(0))

# # train_transform = transforms.Compose([
# #     transforms.ToTensor(),
# #     transforms.Resize((512, 512))])

# # test_transform = transforms.Compose([
# #     transforms.ToTensor(),
# #     transforms.Resize((512, 512))])


# trainloader = DataLoader(train_dataset, 
#                 batch_size = 32, 
#                 shuffle = True,
#                 num_workers = 0,
#                 pin_memory = False)

# testloader = DataLoader(test_dataset,
#                 batch_size = ,
#                 shuffle = False,
#                 num_workers = 0,
#                 pin_memory = False)

In [ ]:
# # get some random training images
# dataiter = iter(trainloader)
# batch = dataiter.next()
# batch['label'].shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def show_image(image, label):

    rgb_img = np.stack((image[2,:,:], image[1,:,:], image[0,:,:]), axis=2).astype(np.uint8)
    gray_img = image[0,:,:][:, :,np.newaxis]
    
    label = np.transpose(label, (1,2,0))
    
    f = plt.figure()
    f.add_subplot(1,2,1)
    plt.imshow(rgb_img)
    
    f.add_subplot(1,2,2)
    plt.imshow(gray_img)

    plt.show(block=True)
    

def display_NAIP(image, label):
    
    # reshape data
    ndata = np.moveaxis(image, 0, -1)  # ndata is of shape (40, 40, 13)
    # select RGB (or any other combination)
    rgb = ndata[...,[2,1,0]]
    
#     nlabel = np.moveaxis(label, 0, -1)
    
    f = plt.figure()
    f.add_subplot(1,2,1)
    plt.imshow(rgb)
    
    f.add_subplot(1,2,2)
    plt.imshow(label)
    plt.show(block=True)


for i in range(20):
    
    sample = dataset[i]
    display_NAIP(**sample)

    if i == 10:
        break

In [8]:
import warnings
from typing import Any, Dict, cast

import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import torch
import torch.nn as nn
from torch import Tensor
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torchmetrics import Accuracy, JaccardIndex, MetricCollection
from sharp_trainer import SemanticSegmentationTask

In [9]:
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.loggers import TensorBoardLogger

In [10]:
conf = OmegaConf.load("./test_multiclass.yaml")
conf_dict = OmegaConf.to_object(conf.experiment)
conf_dict = cast(Dict[Any, Dict[Any, Any]], conf_dict)

# prepare data for training
dl_kwargs = conf_dict["dataloader"]
label_file = dl_kwargs["labelpath"]

# load data
dataset = GenMARSH(label_file, transform=None, normalization=dl_kwargs['normalization'], ndvi=dl_kwargs['ndvi'], ndwi=dl_kwargs['ndwi'], datasource=dl_kwargs['datasource'])

train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(0))

trainloader = DataLoader(train_dataset, 
                batch_size = dl_kwargs["batch_size"], 
                shuffle = True,
                num_workers = dl_kwargs["num_workers"],
                pin_memory = False)

testloader = DataLoader(test_dataset, 
                batch_size = dl_kwargs["batch_size"], 
                shuffle = False,
                num_workers = dl_kwargs["num_workers"],
                pin_memory = False)

# # Setup model
# model_kwargs = conf_dict['module']
    
# experiment_folder = "{}_{}_{}_{}_{}_{}".format(model_kwargs['segmentation_model'], model_kwargs['encoder_name'], model_kwargs['learning_rate'], model_kwargs['in_channels'], model_kwargs['loss'], dl_kwargs['tracker_val'])

    
# experiment_dir = os.path.join(root_path, 'outputs_combined',experiment_folder)
# logger = TensorBoardLogger(experiment_dir, name="models")
    
# checkpoint_callback = ModelCheckpoint(
# monitor=dl_kwargs['tracker_val'], dirpath=experiment_dir, save_top_k=1, save_last=True, mode=dl_kwargs['tracker_mode'])
    
# early_stopping_callback = EarlyStopping(monitor=dl_kwargs['tracker_val'], min_delta=0.00, patience=5, mode=dl_kwargs['tracker_mode'])
    

# model = SemanticSegmentationTask(**model_kwargs)
    
# trainer = pl.Trainer(
#             callbacks=[checkpoint_callback, early_stopping_callback],
#             logger=logger,
#             default_root_dir=experiment_dir,
#             min_epochs=1,
#             max_epochs=100,
#             accelerator="gpu",
#             devices=[2])


In [11]:
monitor_options = ['val_JaccardIndex']
model_options = ['unet', 'fpn', 'pspnet', 'deeplabv3', 'deeplabv3plus', 'pan', 'manet', 'linknet']
encoder_options = ['vgg11']
lr_options = [1e-4]
loss_options = ["ce"]
weight_init_options = ["imagenet"]
in_channel = 4
out_channel = 3

for (model, encoder, lr, loss, weight_init, monitor_state) in itertools.product(
        model_options,
        encoder_options,
        lr_options,
        loss_options,
        weight_init_options,
        monitor_options):
    
    experiment_name = f"{monitor_state}_{model}_{encoder}_{lr}_{loss}_{weight_init}"
    
    print(experiment_name)

    experiment_dir = os.path.join(dl_kwargs["out_dir"], experiment_name)
    logger = TensorBoardLogger(experiment_dir, name="models")
    
    if monitor_state == 'val_loss':
        tracking_mode = 'min'
    elif monitor_state == 'val_JaccardIndex':
        tracking_mode = 'max'
    
    checkpoint_callback = ModelCheckpoint(
        monitor=monitor_state, dirpath=experiment_dir, save_top_k=1, save_last=True, mode=tracking_mode)

    early_stopping_callback = EarlyStopping(monitor=monitor_state, min_delta=0.00, patience=5, mode=tracking_mode)


    model = SemanticSegmentationTask(
                    segmentation_model=model,
                    encoder_name=encoder,
                    encoder_weights=weight_init,
                    learning_rate=lr,
                    in_channels=in_channel,
                    num_classes=out_channel,
                    learning_rate_schedule_patience=6,
                    ignore_index=0,
                    loss=loss,
                    imagenet_pretraining=True)

    trainer = pl.Trainer(
                callbacks=[checkpoint_callback, early_stopping_callback],
                logger=logger,
                default_root_dir=experiment_dir,
                min_epochs=1,
                max_epochs=100,
                accelerator="gpu",
                devices=[3])

    trainer.fit(model, trainloader, testloader)



val_JaccardIndex_unet_vgg11_0.0001_ce_imagenet


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ../outputs_None/val_JaccardIndex_unet_vgg11_0.0001_ce_imagenet/models
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | Unet             | 18.3 M
1 | loss          | CrossEntropyLoss | 0     
2 | train_metrics | MetricCollection | 0     
3 | val_metrics   | MetricCollection | 0     
4 | test_metrics  | MetricCollection | 0     
---------------------------------------------------
18.3 M    Trainable params
0         Non-trainable params
18.3 M    Total params
73.020    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/envs/vims/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


RasterioIOError: /rapids/notebooks/sciclone/geograd/Miranda/github/MarshMapping/data/NAIP/NAIP_image_patches_512/zone8_resample_44544-15872.tif: No such file or directory

In [ ]:
# Load pretrained model

for m in all_model_path:
    
    model_name = os.path.basename(options["model_path"])
        
    if "CW" in model_name: # and "deeplabv3+" not in model_name
        model = SemanticSegmentation.load_from_checkpoint(options["model_path"])
        model.eval()
    else:
        model = SemanticSegmentationTask.load_from_checkpoint(options["model_path"])
        model.eval()
    
    
    
    y_true = []
    y_predicted = []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="testing"):
            
            image = batch["image"]
            target = batch["mask"]

            logits = model(image) # predicted
            logits = torch.nn.functional.softmax(logits, dim=1).argmax(1)
            
            logits = logits.reshape(-1)
            target = target.reshape(-1)

            # Accuracy metrics only on annotated pixels (Check on the dimension changes)

            mask = target != 0 # 0 is the background
            
            logits = logits[mask]
            target = target[mask]

            y_predicted += logits.tolist()
            y_true += target.tolist()
            
        
        ####################################################################
        # Save Scores to the .log file                                     #
        ####################################################################
        acc = Evaluation(y_predicted, y_true)
        logging.info("\n")
        logging.info("STATISTICS: \n")
        logging.info("Evaluation: " + str(acc))
        print("Evaluation: " + str(acc))
        """
        Confusion matrix whose i-th row and j-th column entry indicates the number of samples with true label 
        You being i-th class and predicted label being j-th class.
        """
        conf_mat = confusion_matrix(y_true, y_predicted, options['labels'])
        
        logging.info("Confusion Matrix:  \n" + str(conf_mat.to_string()))
        print("Confusion Matrix:  \n" + str(conf_mat.to_string()))
 

        if options['predict_masks']:
            
            path = os.path.join(up(up(root_path)), 'VIMS', 'NAIP', 'VA_NAIP_2018_8977', 'test')
            ROIs = [file for file in os.listdir(path) if file.split('.')[-1]=='tif']

            impute_nan = np.tile(bands_mean, (256,256,1))
                        
            for roi in tqdm(ROIs):
                
                roi_file = os.path.join(path, roi)
                os.makedirs(options['gen_masks_path'], exist_ok=True)
            
                output_image = os.path.join(options['gen_masks_path'], os.path.basename(roi_file).split('.tif')[0] + '_unet.tif')
            
                # Read metadata of the initial image
                with rasterio.open(roi_file, mode ='r') as src:
                    tags = src.tags().copy()
                    meta = src.meta
                    image = src.read()
                    image = np.moveaxis(image, (0, 1, 2), (2, 0, 1)).astype('float32')
                    dtype = src.read(1).dtype
            
                # Update meta to reflect the number of layers
                meta.update(count = 1)
            
                # Write it
                with rasterio.open(output_image, 'w', **meta) as dst:
                    
                    # Preprocessing before prediction
                    nan_mask = np.isnan(image)
                    image[nan_mask] = impute_nan[nan_mask]
                
                    image = transform_test(image)
                    image = standardization(image)

                    # Predictions
                    logits = model(image.unsqueeze(0))
                    probs = logits.softmax(dim=1).numpy().argmax(1).squeeze()
                    
                    # Write the mask with georeference
                    dst.write_band(1, probs.astype(dtype).copy()) # In order to be in the same dtype
                    dst.update_tags(**tags)


In [ ]:
# test_loader = DataLoader(test_dataset, 
#                 batch_size = 32, 
#                 shuffle = False,
#                 num_workers = 0,
#                 pin_memory = False)

In [ ]:


for m in os.listdir(os.path.join(root_path, 'outputs')):
    if not m.endswith('ipynb_checkpoints'):
        print("Working on {}".format(m))
        start_time = time.time()
        mpath = os.path.join(root_path, 'outputs', m, 'last.ckpt')
        model_load = SemanticSegmentationTask.load_from_checkpoint(mpath)
        model_load.eval()
        
        y_true = []
        y_predicted = []
        
        with torch.no_grad():

            for batch in testloader:

                image = batch["image"]
                target = batch["label"]

                logits = model_load(image) # predicted
                logits = torch.nn.functional.softmax(logits, dim=1).argmax(1)

                logits = logits.reshape(-1)
                target = target.reshape(-1)

                # Accuracy metrics only on annotated pixels (Check on the dimension changes)

                mask = target != 0 # 0 is the background
                logits = logits[mask]
                target = target[mask]

                y_predicted += logits.tolist()
                y_true += target.tolist()
        
        acc = metrics.Evaluation(y_predicted, y_true)
        print("Evaluation: " + str(acc))
        """
        Confusion matrix whose i-th row and j-th column entry indicates the number of samples with true label 
        You being i-th class and predicted label being j-th class.
        """
        try:
            test_labels = ['high_marsh', 'low_marsh']
            conf_mat = metrics.confusion_matrix(y_true, y_predicted, test_labels)
            print("Confusion Matrix:  \n" + str(conf_mat.to_string()))
        except:
            labels = ['Background', 'high_marsh', 'low_marsh']
            conf_mat = metrics.confusion_matrix(y_true, y_predicted, labels)
            print("Confusion Matrix:  \n" + str(conf_mat.to_string()))
            

        end_time = time.time()
        time_diff = end_time - start_time
        print("time spent: {}".format(time_diff))
